<a href="https://colab.research.google.com/github/JeanCarloBejaran/Qualia-Nootropics-Mind-vs.-Focus/blob/master/Qualia_Nootropics_Mind_Vs_Focus_Ingredient_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Qualia Nootropics released a new product (Qualia Focus) which appears to be very similar to Qualia mind. But what are the resulting differences? Should I switch to Focus or stay with Mind?** We will be solving this through data-driven decision-making. 


From available information, Qualia Mind seems to be about energy and focus, and memory, but also brain health, protection and health. Focus works on delivering short-term benefits for half the price" Instead of blindly trusting marketing, let's analyze the ingredients to obtain all the information for some data-driven decision making.

#Ingredients Tradeoff Analysis

In [0]:
import pandas as pd

In [0]:
#To-do: Get ingredients list for each formulation from their website.
Q_Mind_url = 'https://neurohacker.com/shop/qualia-mind'
Q_Focus_url = 'https://neurohacker.com/shop/qualia-focus'

'''
Q_Mind = pd.read_csv(Q_Mind_url)
Q_Focus = pd.read_csv(Q_Focus_url)

This approach gives the following error.

HTTPError                                 Traceback (most recent call last)
<ipython-input-196-18599c8e1d8c> in <module>()
      2 Q_Focus_url = 'https://neurohacker.com/shop/qualia-focus'
      3 
----> 4 Q_Mind = pd.read_csv(Q_Mind_url)
      5 Q_Focus = pd.read_csv(Q_Focus_url)

8 frames
/usr/lib/python3.6/urllib/request.py in http_error_default(self, req, fp, code, msg, hdrs)
    648 class HTTPDefaultErrorHandler(BaseHandler):
    649     def http_error_default(self, req, fp, code, msg, hdrs):
--> 650         raise HTTPError(req.full_url, code, msg, hdrs, fp)
    651 
    652 class HTTPRedirectHandler(BaseHandler):

HTTPError: HTTP Error 403: Forbidden

'''

"\nQ_Mind = pd.read_csv(Q_Mind_url)\nQ_Focus = pd.read_csv(Q_Focus_url)\n\nThis approach gives the following error.\n\nHTTPError                                 Traceback (most recent call last)\n<ipython-input-196-18599c8e1d8c> in <module>()\n      2 Q_Focus_url = 'https://neurohacker.com/shop/qualia-focus'\n      3 \n----> 4 Q_Mind = pd.read_csv(Q_Mind_url)\n      5 Q_Focus = pd.read_csv(Q_Focus_url)\n\n8 frames\n/usr/lib/python3.6/urllib/request.py in http_error_default(self, req, fp, code, msg, hdrs)\n    648 class HTTPDefaultErrorHandler(BaseHandler):\n    649     def http_error_default(self, req, fp, code, msg, hdrs):\n--> 650         raise HTTPError(req.full_url, code, msg, hdrs, fp)\n    651 \n    652 class HTTPRedirectHandler(BaseHandler):\n\nHTTPError: HTTP Error 403: Forbidden\n\n"

In [0]:
#Error with pd.read_html() -> HTTPError: HTTP Error 403: Forbidden
#Solution: make the server believe a browser is requesting the information
import requests

def get_data(url):
  ''' request data making the server believe a browser is requesting the information'''
  header = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest"
          }

  r = requests.get(url, headers=header)
  df = pd.read_html(r.text)

  return df[0]


In [0]:
# Now get the data
Qualia_Mind_raw = get_data(Q_Mind_url)
Qualia_Focus_raw = get_data(Q_Focus_url)

In [0]:
Qualia_Focus_raw.head(3)

,Amount per Serving,%DV,Unnamed: 2
0,Vitamin C (as ascorbic acid) 100 mg,111%,NaN
1,"Vitamin D3 (as cholecalciferol) 25 mcg ( 1,000...",125%,NaN
2,Thiamine (as thiamine HCI) 50 mg,4167%,NaN


In [0]:
Qualia_Mind_raw.head(3)

,Amount per Serving,%DV,Unnamed: 2
0,Vitamin C (as ascorbic acid) 100 mg,111%,NaN
1,"Vitamin D3 (as cholecalciferol) 25 mcg ( 1,000...",125%,NaN
2,Thiamine (as thiamine HCI) 50 mg,4167%,NaN


In [0]:
#Separate miligrams
Qualia_Mind_raw['Mind_mg/serving'] = Qualia_Mind_raw['Amount per Serving'].apply(lambda x: x[-6:-2])
#repeat for focus
Qualia_Focus_raw['Focus_mg/serving'] = Qualia_Focus_raw['Amount per Serving'].apply(lambda x: x[-6:-2])

Qualia_Mind_raw.head(3)

,Amount per Serving,%DV,Unnamed: 2,Mind_mg/serving
0,Vitamin C (as ascorbic acid) 100 mg,111%,NaN,100
1,"Vitamin D3 (as cholecalciferol) 25 mcg ( 1,000...",125%,NaN,00 I
2,Thiamine (as thiamine HCI) 50 mg,4167%,NaN,50


In [0]:
Qualia_Focus_raw.head(3)

,Amount per Serving,%DV,Unnamed: 2,Focus_mg/serving
0,Vitamin C (as ascorbic acid) 100 mg,111%,NaN,100
1,"Vitamin D3 (as cholecalciferol) 25 mcg ( 1,000...",125%,NaN,00 I
2,Thiamine (as thiamine HCI) 50 mg,4167%,NaN,50


In [0]:
#Clean Data: #Remove last row of both, columns [1,2], remove base stack from both (rows[0:5]), clean the amount miligrams and remove them from Ingredient's label. 
Qualia_Mind = pd.DataFrame( data = Qualia_Mind_raw, columns = Qualia_Mind_raw.columns[[0,3]])
Qualia_Focus = pd.DataFrame( data = Qualia_Focus_raw, columns = Qualia_Focus_raw.columns[[0,3]])

#rename first column
Qualia_Mind.rename(columns = {Qualia_Mind.columns[0]: "Mind_ingredients"}, inplace = True)
Qualia_Focus.rename(columns = {Qualia_Focus.columns[0]: "Focus_ingredients"}, inplace = True)

Qualia_Mind.head(7)

,Mind_ingredients,Mind_mg/serving
0,Vitamin C (as ascorbic acid) 100 mg,100
1,"Vitamin D3 (as cholecalciferol) 25 mcg ( 1,000...",00 I
2,Thiamine (as thiamine HCI) 50 mg,50
3,Niacin (as niacinamide) 50 mg,50
4,Vitamin B6 (as pyridoxal 5' -phosphate) 20 mg,20
5,Vitamin B12 (as methylcobalamin) 1000 mcg,00 m
6,Pantothenic Acid (as calcium pantothenate) 50 mg,50


In [0]:
Qualia_Focus.head(7)

,Focus_ingredients,Focus_mg/serving
0,Vitamin C (as ascorbic acid) 100 mg,100
1,"Vitamin D3 (as cholecalciferol) 25 mcg ( 1,000...",00 I
2,Thiamine (as thiamine HCI) 50 mg,50
3,Niacin (as niacinamide) 50 mg,50
4,Vitamin B6 (as pyridoxal 5' -phosphate) 20 mg,20
5,Vitamin B12 (as methylcobalamin) 1000 mcg,00 m
6,Pantothenic Acid (as calcium pantothenate) 50 mg,50


We can remove the first rows of both columns since they are the same and we are interested in the differences. 

In [0]:
#Drop Base Stack Ingredients
Qualia_Mind = Qualia_Mind[7:-1]
Qualia_Focus = Qualia_Focus[7:-1]

In [0]:
Qualia_Mind.head(3)

,Mind_ingredients,Mind_mg/serving
7,Acetyl-L-Carnitine HCI 500 mg,500
8,Artichoke Leaf Extract (4% cynarin and related...,500
9,Bacopa monnieri Leaf Extract 300 mg,300


In [0]:
Qualia_Focus.head(3)

,Focus_ingredients,Focus_mg/serving
7,Artichoke Leaf Extract (4% cynarin and related...,300
8,Bacopa monnieri Leaf Extract 300 mg,300
9,Alpha-Glycerylphosphorylcholine (alpha GPC) 30...,300


In [0]:
#Clean amount(mg) column and make it type integer.
Qualia_Mind['Mind_mg/serving'] = Qualia_Mind[Qualia_Mind.columns[1]].apply(lambda s: int([''.join(i for i in s if i.isdigit())][0]))
Qualia_Focus['Focus_mg/serving'] = Qualia_Focus[Qualia_Focus.columns[1]].apply(lambda s: int([''.join(i for i in s if i.isdigit())][0]))
Qualia_Mind

,Mind_ingredients,Mind_mg/serving
7,Acetyl-L-Carnitine HCI 500 mg,500
8,Artichoke Leaf Extract (4% cynarin and related...,500
9,Bacopa monnieri Leaf Extract 300 mg,300
10,Rhodiola rosea Root Extract (3% rosavins; 1% s...,300
11,DL-Phenylalanine 300 mg,300
12,Uridine-5' -Monophosphate heptahydrate disodiu...,250
13,N-Acetyl-L-Tyrosine 250 mg,250
14,Taurine 200 mg,200
15,L-Theanine 200 mg,200
16,Alpha-Glycerylphosphorylcholine (alpha GPC) 20...,200


In [0]:
Qualia_Focus

,Focus_ingredients,Focus_mg/serving
7,Artichoke Leaf Extract (4% cynarin and related...,300
8,Bacopa monnieri Leaf Extract 300 mg,300
9,Alpha-Glycerylphosphorylcholine (alpha GPC) 30...,300
10,DL-Phenylalanine 300 mg,300
11,Acetyl-L-Carnitine HCI 250 mg,250
12,N-Acetyl-L-Tyrosine 250 mg,250
13,Uridine-5' -Monophosphate heptahydrate disodiu...,200
14,Taurine 200 mg,200
15,L-Theanine 200 mg,200
16,Rhodiola rosea Root Extract (3% rosavins; 1% s...,150


In [0]:
Qualia_Mind['merge_key'] = Qualia_Mind.Mind_ingredients.apply(lambda x: x[0:10])
Qualia_Focus['merge_key'] = Qualia_Focus.Focus_ingredients.apply(lambda x: x[0:10])

In [0]:
#merge on ingredient name
Qualia = pd.merge(Qualia_Mind, Qualia_Focus, how = 'outer', on = 'merge_key')
Qualia.index = Qualia.merge_key
Qualia.drop(columns = ['merge_key', 'Focus_ingredients'], inplace = True)

Qualia.rename(columns = {Qualia.columns[0]: "Ingredients"}, inplace = True)

In [0]:
Qualia

,Ingredients,Mind_mg/serving,Focus_mg/serving
merge_key,,,
Acetyl-L-C,Acetyl-L-Carnitine HCI 500 mg,500,250.0
Artichoke,Artichoke Leaf Extract (4% cynarin and related...,500,300.0
Bacopa mon,Bacopa monnieri Leaf Extract 300 mg,300,300.0
Rhodiola r,Rhodiola rosea Root Extract (3% rosavins; 1% s...,300,150.0
DL-Phenyla,DL-Phenylalanine 300 mg,300,300.0
Uridine-5',Uridine-5' -Monophosphate heptahydrate disodiu...,250,200.0
N-Acetyl-L,N-Acetyl-L-Tyrosine 250 mg,250,250.0
Taurine 20,Taurine 200 mg,200,200.0
L-Theanine,L-Theanine 200 mg,200,200.0


#clean ingredients 


In [0]:
#clean ingredients list of the mg amounts
#remove ingredients with the same quantity. 9 ingredients remain. 4 completely removed, 5 with altered quantities. 

To-Do: 

1. What are the ingredients in Mind but not in Focus? What do they cause (effect)?
2. What are the differences in mg and percentages in the ingredients of each formulation?
3. Remove ingredients that are equally present and get the effect info for the remaining.


In [0]:
#find the difference with each ingredient, Add a difference column printing Focus has more X / Qualia +50mg
#pandas diff() and pct_diff()
#create a category for each effect and quantify assuming it is dependant on mgs / serving. 

In [0]:
#Visualize the data to observe the differences in ingredients with different quantities.
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px
#https://plot.ly/python/histograms/

In [0]:
#See effect of lower dosages of Qualia Mind vs. Qualia Focus full dosage + DHA in total price

1. Mine the overview for each ingredient from neurohackers website.
2. Merge all ti a final dataset.
3. Use NLTK to summarize main effects into tokenized clusters found in other clusters. 
Then Update the table with this information added.

In [0]:
#Ex: Artichoke's Overview
# Artichoke (Cynara cardunculus) is a plant that contains cynarin, having nootropic effects. Cynarin can significantly improve memory and executive function.